## Import libraries

In [30]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [31]:
train_data = np.load('dataset/Trousers & Jeans - Female - train.npz')
validation_data = np.load('dataset/Trousers & Jeans - Female - validation.npz')
test_data = np.load('dataset/Trousers & Jeans - Female - test.npz')

In [32]:
np.unique(train_data)

array(['images', 'labels'], dtype='<U6')

In [33]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [34]:
np.unique(labels_train)

array([0, 1])

In [35]:
images_train.shape

(2002, 120, 90, 3)

In [36]:
labels_train.shape

(2002,)

In [37]:
images_validation.shape

(250, 120, 90, 3)

In [38]:
labels_validation.shape

(250,)

In [39]:
images_test.shape

(250, 120, 90, 3)

In [40]:
labels_test.shape

(250,)

In [41]:
# standardize 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create model

### Define some hyperparameters

In [42]:
EPOCHS = 15
BATCH_SIZE = 64

In [43]:
HP_KERNEL_SIZE_1 = hp.HParam('kernel size 1', hp.Discrete([3,7]))
HP_KERNEL_SIZE_2 = hp.HParam('kernel size 2', hp.Discrete([3,5]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64,128]))
HP_DENSE_SIZE = hp.HParam('dense size', hp.Discrete([256,512,1024]))

METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1 (Trousers&Jeans_Female)/hparam_tuning').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE_1,HP_KERNEL_SIZE_2,HP_KERNEL_NUM,HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [44]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_1], activation = 'relu', input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_2], activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(2, activation = 'softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 1 (Trousers&Jeans_Female)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers', 'Jeans'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, profile_batch = 0)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 1 (Trousers&Jeans_Female)\Run-{}'.format(session_num))
    
    return accuracy

In [45]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [46]:
session_num = 1
for kernel_size_1 in HP_KERNEL_SIZE_1.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for kernel_size_2 in HP_KERNEL_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
                hparams = {
                    HP_KERNEL_SIZE_1 : kernel_size_1,
                    HP_KERNEL_SIZE_2 : kernel_size_2,
                    HP_KERNEL_NUM : kernel_num,
                    HP_DENSE_SIZE : dense_size
                }
                run_name = f'run-{session_num}'
                print('--- Starting Trial:',run_name)
                print({h.name: hparams[h] for h in hparams})
                run("Logs/Model 1 (Trousers&Jeans_Female)/hparam_tuning/" + run_name, hparams, session_num)

                session_num += 1

--- Starting Trial: run-1
{'kernel size 1': 3, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 256}
Epoch 1/15
32/32 [==============================] - 12s 380ms/step - loss: 0.8185 - accuracy: 0.6828 - val_loss: 0.5150 - val_accuracy: 0.7880
Epoch 2/15
32/32 [==============================] - 11s 356ms/step - loss: 0.4774 - accuracy: 0.7577 - val_loss: 0.5367 - val_accuracy: 0.7040
Epoch 3/15
32/32 [==============================] - 10s 320ms/step - loss: 0.4340 - accuracy: 0.7792 - val_loss: 0.4212 - val_accuracy: 0.8040
Epoch 4/15
32/32 [==============================] - 10s 314ms/step - loss: 0.4065 - accuracy: 0.7977 - val_loss: 0.4342 - val_accuracy: 0.8120
Epoch 5/15
8/8 [==============================] - 0s 39ms/step - loss: 0.4212 - accuracy: 0.8040
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automati

8/8 [==============================] - 0s 53ms/step - loss: 0.4384 - accuracy: 0.7800
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Female)\Run-5\assets
--- Starting Trial: run-6
{'kernel size 1': 3, 'kernel size 2': 5, 'kernel num': 32, 'dense size': 1024}
Epoch 1/15
32/32 [==============================] - 18s 548ms/step - loss: 1.0699 - accuracy: 0.6613 - val_loss: 0.6781 - val_accuracy: 0.6560
Epoch 2/15
32/32 [==============================] - 18s 549ms/step - loss: 0.4986 - accuracy: 0.7582 - val_loss: 0.4929 - val_accuracy: 0.7440
Epoch 3/15
32/32 [==============================] - 17s 536ms/step - loss: 0.4594 - accuracy: 0.7712 - val_loss: 0.4518 - val_accuracy: 0.7680
Epoch 4/15
32/32 [==============================] - 17s 537ms/step - loss: 0.4494 - accuracy: 0.7717 - val_loss: 0.4507 - val_accuracy: 0.7840
Epoch 5/15
32/32 [==============================] - 19s 585ms/step - loss: 0.4249 - accuracy: 0.7847 - val_loss: 0.4278 - val_accuracy: 0.7920
E

Epoch 3/15
32/32 [==============================] - 48s 1s/step - loss: 0.4666 - accuracy: 0.7647 - val_loss: 0.4826 - val_accuracy: 0.7760
Epoch 4/15
32/32 [==============================] - 48s 2s/step - loss: 0.4225 - accuracy: 0.7882 - val_loss: 0.4685 - val_accuracy: 0.7400
Epoch 5/15
32/32 [==============================] - 55s 2s/step - loss: 0.4038 - accuracy: 0.7947 - val_loss: 0.4992 - val_accuracy: 0.7400
Epoch 6/15
32/32 [==============================] - 50s 2s/step - loss: 0.3832 - accuracy: 0.8112 - val_loss: 0.4375 - val_accuracy: 0.7880
Epoch 7/15
32/32 [==============================] - 48s 1s/step - loss: 0.3285 - accuracy: 0.8367 - val_loss: 0.4140 - val_accuracy: 0.7920
Epoch 8/15
32/32 [==============================] - 47s 1s/step - loss: 0.3364 - accuracy: 0.8447 - val_loss: 0.4900 - val_accuracy: 0.7960
Epoch 9/15
32/32 [==============================] - 48s 1s/step - loss: 0.3106 - accuracy: 0.8497 - val_loss: 0.4021 - val_accuracy: 0.8200
Epoch 10/15
32/32 [=

--- Starting Trial: run-19
{'kernel size 1': 7, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 256}
Epoch 1/15
32/32 [==============================] - 15s 470ms/step - loss: 0.8734 - accuracy: 0.6199 - val_loss: 0.5764 - val_accuracy: 0.6880
Epoch 2/15
32/32 [==============================] - 15s 475ms/step - loss: 0.5429 - accuracy: 0.7178 - val_loss: 0.5410 - val_accuracy: 0.7840
Epoch 3/15
32/32 [==============================] - 14s 450ms/step - loss: 0.4899 - accuracy: 0.7527 - val_loss: 0.4933 - val_accuracy: 0.7560
Epoch 4/15
32/32 [==============================] - 15s 473ms/step - loss: 0.4612 - accuracy: 0.7722 - val_loss: 0.4942 - val_accuracy: 0.7400
Epoch 5/15
8/8 [==============================] - 0s 51ms/step - loss: 0.4933 - accuracy: 0.7560
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Female)\Run-19\assets
--- Starting Trial: run-20
{'kernel size 1': 7, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 512}
Epoch 1/15
32/32 [=========

32/32 [==============================] - 24s 747ms/step - loss: 0.5477 - accuracy: 0.7168 - val_loss: 0.5750 - val_accuracy: 0.7440
Epoch 3/15
32/32 [==============================] - 24s 751ms/step - loss: 0.5164 - accuracy: 0.7233 - val_loss: 0.5158 - val_accuracy: 0.7800
Epoch 4/15
32/32 [==============================] - 24s 752ms/step - loss: 0.4955 - accuracy: 0.7448 - val_loss: 0.4859 - val_accuracy: 0.7200
Epoch 5/15
32/32 [==============================] - 24s 752ms/step - loss: 0.4695 - accuracy: 0.7647 - val_loss: 0.4652 - val_accuracy: 0.8040
Epoch 6/15
32/32 [==============================] - 24s 744ms/step - loss: 0.4370 - accuracy: 0.7832 - val_loss: 0.5096 - val_accuracy: 0.7200
Epoch 7/15
8/8 [==============================] - 1s 86ms/step - loss: 0.4652 - accuracy: 0.8040
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Female)\Run-25\assets
--- Starting Trial: run-26
{'kernel size 1': 7, 'kernel size 2': 3, 'kernel num': 64, 'dense size': 512}


Epoch 7/15
32/32 [==============================] - 47s 1s/step - loss: 0.4717 - accuracy: 0.7627 - val_loss: 0.4960 - val_accuracy: 0.7360
Epoch 8/15
32/32 [==============================] - 47s 1s/step - loss: 0.4526 - accuracy: 0.7682 - val_loss: 0.4804 - val_accuracy: 0.7320
Epoch 9/15
32/32 [==============================] - 47s 1s/step - loss: 0.4869 - accuracy: 0.7602 - val_loss: 0.4843 - val_accuracy: 0.7480
Epoch 10/15
32/32 [==============================] - 46s 1s/step - loss: 0.4328 - accuracy: 0.7802 - val_loss: 0.4719 - val_accuracy: 0.7640
Epoch 11/15
32/32 [==============================] - 47s 1s/step - loss: 0.4256 - accuracy: 0.7872 - val_loss: 0.5094 - val_accuracy: 0.7680
Epoch 12/15
32/32 [==============================] - 47s 1s/step - loss: 0.4326 - accuracy: 0.7782 - val_loss: 0.4651 - val_accuracy: 0.7560
Epoch 13/15
32/32 [==============================] - 47s 1s/step - loss: 0.4029 - accuracy: 0.7932 - val_loss: 0.4523 - val_accuracy: 0.7680
Epoch 14/15
32/3

8/8 [==============================] - 3s 373ms/step - loss: 0.5144 - accuracy: 0.7600
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Female)\Run-36\assets
